In [1]:
from cen.configs.config import ExperimentConfig
from cen.run import run_experiment

# Mnist

## Baseline
### run_baseline.sh
```console
PROBLEM=mnist

CUDA_VISIBLE_DEVICES=0 \
python -u -m cen.run \
  experiment=train_eval \
  problem=${PROBLEM} \
  encoder=${PROBLEM}/cnn \
  model=${PROBLEM}/baseline \
  optimizer=adam \
  run.seed=1
```

In [2]:
experiment_config_as_dict = {
    'run': {
        'seed': 1
    },
    'crossval': None,
    'dataset': {
        'context_kwargs': {
            'datapath': 'mnist.npz',
            'standardize': False
        },
        'feature_kwargs': {
            'datapath': 'mnist.interp.hog3x3.npz',
            'feature_type': 'hog3x3',
            'remove_const_features': True,
            'signal_to_noise': None,
            'standardize': True,
            'whiten': False
        },
        'name': 'mnist'
    },
    'evaluation': {
        'batch_size': 512,
        'metrics': {
            'acc': {
                'name': 'categorical_accuracy'
            }
        }
    },
    'model': {
        'kwargs': {
            'top_dense_layers': 1,
            'top_dense_units': 256,
            'top_dense_activation': 'relu',
            'top_dense_dropout': 0.5
        },
        'name': 'baseline'
    },
    'network': {
        'kwargs': {
            'name': 'simple_cnn',
            'blocks': 1,
            'filters': 64,
            'dropout': 0.2
        }
    },
    'optimizer': {
        'class_name': 'adam',
        'config': {
            'amsgrad': True,
            'learning_rate': 3e-3,
            'decay': 1e-4
        }
    },
    'train': {
        'batch_size': 512,
        'checkpoint_kwargs': {
            'monitor': 'val_categorical_accuracy',
            'save_best_only': True,
            'save_weights_only': True,
            'verbose': True
        },
        'epochs': 10,#100,
        'loss': {
            'name': 'categorical_crossentropy'
        },
        'shuffle_buffer_size': 4096,
        'tensorboard': {
            'histogram_freq': 1,
            'write_graph': False,
            'write_images': False,
            'update_freq': 'batch'
        }
    }
}
experiment_config = ExperimentConfig.from_dict(experiment_config_as_dict)

In [3]:
run_experiment(experiment_config)

Epoch 1/10


/home/stevengt/anaconda3/envs/cen/lib/python3.7/site-packages/tensorflow/python/keras/engine/functional.py:595: UserWarning: Input dict contained keys ['X'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


98/98 [==============================] - 84s 854ms/step - loss: 0.5865 - categorical_accuracy: 0.8034 - val_loss: 0.0581 - val_categorical_accuracy: 0.9836

Epoch 00001: val_categorical_accuracy improved from -inf to 0.98360, saving model to /home/stevengt/Documents/code/cen/examples/mnist/checkpoint
Epoch 2/10
98/98 [==============================] - 77s 786ms/step - loss: 0.0772 - categorical_accuracy: 0.9761 - val_loss: 0.0473 - val_categorical_accuracy: 0.9852

Epoch 00002: val_categorical_accuracy improved from 0.98360 to 0.98520, saving model to /home/stevengt/Documents/code/cen/examples/mnist/checkpoint
Epoch 3/10
98/98 [==============================] - 78s 794ms/step - loss: 0.0540 - categorical_accuracy: 0.9830 - val_loss: 0.0371 - val_categorical_accuracy: 0.9899

Epoch 00003: val_categorical_accuracy improved from 0.98520 to 0.98990, saving model to /home/stevengt/Documents/code/cen/examples/mnist/checkpoint
Epoch 4/10
98/98 [==============================] - 75s 761ms/step

## CEN

### run_cen.sh
```console
PROBLEM=mnist

CUDA_VISIBLE_DEVICES=0 \
python -u -m cen.run \
  experiment=train_eval \
  problem=${PROBLEM} \
  encoder=${PROBLEM}/cnn \
  model=${PROBLEM}/cen_convex_hog \
  optimizer=adam \
  run.seed=1
```

In [6]:
experiment_config_as_dict['model'] = {
    'kwargs': {
        'explainer_name': 'convex',
        'explainer_kwargs': {
            'dict_size': 128,
            'use_bias': True,
            'entropy_regularizer': {
                'class_name': 'ctx_cond_neg_ent',
                'config': {
                    'coeff': 2e-2,
                    'num_samples': 256,
                    'stddev': 0.2
                }
            },
            'kernel_regularizer': {
                'class_name': 'l1_l2',
                'config': {
                    'l1': 1e-4,
                    'l2': 1e-5
                }
            },
            'dict_kernel_regularizer': {
                'class_name': 'l1_l2',
                'config': {
                    'l1': 1e-4,
                    'l2': 1e-5
                }
            }
        },
        'top_dense_layers': 1,
        'top_dense_units': 256,
        'top_dense_activation': 'relu',
        'top_dense_dropout': 0.5
    },
    'name': 'cen'
}

experiment_config = ExperimentConfig.from_dict(experiment_config_as_dict)

In [7]:
run_experiment(experiment_config)

Epoch 1/10
98/98 [==============================] - 167s 2s/step - loss: 2.6173 - categorical_accuracy: 0.7813 - val_loss: 0.6697 - val_categorical_accuracy: 0.9635

Epoch 00001: val_categorical_accuracy improved from -inf to 0.96350, saving model to /home/stevengt/Documents/code/cen/examples/mnist/checkpoint
Epoch 2/10
98/98 [==============================] - 165s 2s/step - loss: 0.6548 - categorical_accuracy: 0.9537 - val_loss: 0.4714 - val_categorical_accuracy: 0.9764

Epoch 00002: val_categorical_accuracy improved from 0.96350 to 0.97640, saving model to /home/stevengt/Documents/code/cen/examples/mnist/checkpoint
Epoch 3/10
98/98 [==============================] - 156s 2s/step - loss: 0.4907 - categorical_accuracy: 0.9679 - val_loss: 0.3902 - val_categorical_accuracy: 0.9824

Epoch 00003: val_categorical_accuracy improved from 0.97640 to 0.98240, saving model to /home/stevengt/Documents/code/cen/examples/mnist/checkpoint
Epoch 4/10
98/98 [==============================] - 156s 2s/s